<a href="https://colab.research.google.com/github/watanabe-gk/study_gpt/blob/stg/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ライブラリーの利用
!git clone https://github.com/Michi-123/whiteGPT

fatal: destination path 'whiteGPT' already exists and is not an empty directory.


https://en.wikipedia.org/wiki/Word2vec

In [ ]:
#@title ライブラリーの読み込み
import random
import torch
import torch.nn as nn
import torch.optim as optim
from whiteGPT import word2vec
from whiteGPT import CBOW
from whiteGPT import Vocab

In [ ]:
# @title 学習用のテキストデータ
corpus_list = [
    "this animal is cat.",
    "the quick brown fox jumps over the lazy dog.",
    "dog and cat are animal.",
    "I love dogs and cats.",
    "the dog sat on the rug.",
    "cats are independent animals. ",
    "foxes are wild animals.",
    "The cat prowled through the moonlit garden.",
    "Dogs eagerly awaited their owner's return at the doorstep.",
    "A cat's purr filled the room with comfort.",
    "The dog wagged its tail in excitement.",
    "Cats gracefully leaped from rooftop to rooftop.",

    "Walking down the street, I spotted a stray dog searching for scraps.",
    "The cat stretched lazily in the warmth of the sunbeam.",
    "Dogs barked joyfully in the park.",
    "A sleek black cat slinked along the fence.",
    "The old dog snoozed contentedly by the fireplace.",
    "Cats darted through the alleyways, chasing shadows.",
    "A fluffy white cat napped peacefully on the windowsill.",

    "Dogs are furry friends who love to play fetch and cuddle with you.",
    "Cats are soft and independent pets that enjoy lounging in sunny spots.",
    "Dogs wag their tails when they're happy and bark to say hello.",
    "Cats purr when they're content and love to curl up in your lap.",

    "Fido loves going for car rides, he always sticks his head out the window with a goofy grin.",
    "the old dog hobbled over to greet us, his tail thumping gently against the floor.",
    "the cat perched regally on the windowsill, surveying its outdoor kingdom. ",
    "the cat stalked a dust bunny across the floor, pouncing with laser focus. ",
    "the neighbor's cat, notorious for its thievery, snuck into our yard and made off with a shiny red ball of yarn. ",
    "curled up with a good book, I felt a soft nudge – my cat, wanting some attention, was rubbing against my leg. ",
    "the therapy dog, with its gentle demeanor, brought a wave of calm to the anxious patients in the waiting room.",
    "despite their different personalities, the dog and cat often napped curled up together. ",
    "we need to buy more cat food, Whiskers seems to be inhaling everything in the bow. ",
    "despite being a scaredy cat, Luna the ginger bravely explored every corner of the new house. ",
    "Max the dog spent all afternoon digging a hole in the backyard, much to the gardener's dismay. ",
    "the smell of freshly baked cookies lured the cat out from its hiding spot. ",
    "during thunderstorms, Milo the dog would huddle under the bed, trembling uncontrollably. ",
    "we adopted a pair of playful kittens, and now our living room is a whirlwind of fur and feathery toys. ",
    "every morning, the rooster crows and the dog barks, creating a chaotic symphony to wake up the household. "
]

In [ ]:
# @title テスト用のテキストデータ
test_corpus_list = [
    "this animal is dog ",
    "the old cat hobbled ",
    "the dog perched regally ",
    "quick brown cat jumps ",
    "cat and dog often ",
    "the dog out from ",
    "Fido the cat would ",
    "Luna the cat spent ",
    "neighbor's dog ",
    "cat stalked a dust ",
    "buy more cat food",
    "cat and the dog barks",
]

# CBOWモデルの学習

In [ ]:
# @title ハイパー・パラメータの設定
# @markdown 埋め込みの次元の数
embedding_dim = 16 #@param{type:'integer'}
# @markdown エポック数
num_epoch = 300 #@param{type:'integer'}
# @markdown 学習係数（学習率）
learning_rate = 0.001 #@param{type:'number'}
# @markdown 入力枠
window_size = 3 #@param{type:'integer'}
# word2vec.window_size = window_size

In [ ]:
# @title インスタンス化
# 学習しやすいデータに加工します
train_corpus = word2vec.modify(corpus_list, window_size)

# Dataset
vocab = Vocab(train_corpus)
train_dataset = word2vec.TextDataset(vocab, train_corpus, window_size)
train_dataloader = word2vec.DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataset = word2vec.TextDataset(vocab, '', window_size)
# 検証しやすいデータを設定します
test_dataset.test_corpus(test_corpus_list)

# 損失関数の計算
criterion = nn.CrossEntropyLoss()
# モデルのインスタンス化
model = word2vec.CBOW(vocab.vocab_size, embedding_dim)
# 最適化モジュール
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 語彙数の確認


In [ ]:
# debug
train_iter = iter(train_dataloader)
next(train_iter)

{'source': tensor([[197,   8,   9],
         [169,  10,  76],
         [ 93,  24, 257],
         [  9, 169,  10]]),
 'target': tensor([169, 149,  22,  11])}

In [ ]:
# @title 学習の実行
for epoch in range(num_epoch):
    total_loss = 0
    running_loss = 0.0
    for batch in train_dataloader:

        # 入力データと教師データの取得
        context_indices, target_index = batch['source'], batch['target']
        # 推論
        output = model(context_indices)
        # 損失計算
        loss = criterion(output, target_index)

        # 勾配の初期化
        optimizer.zero_grad()
        # 勾配の計算
        loss.backward()
        # 学習パラメータ（重み）の更新
        optimizer.step()
        # 損失の累計
        running_loss += loss.item()

    # 損失の平均
    total_loss = running_loss / len(train_dataloader)

    # n回に一度の処理
    if epoch % 10 == 0:
        # ログの出力
        print(f'Epoch {epoch}, Loss: {total_loss:4f}')
        # テスト
        test_dataset.test(model)

Epoch 0, Loss: 5.160475
this animal is : dog : spots
Epoch 10, Loss: 3.190813
the dog out : from : .
Epoch 20, Loss: 2.581051
the dog perched : regally : .
Epoch 30, Loss: 2.090761
quick brown cat : jumps : in
Epoch 40, Loss: 1.705437
the dog out : from : its
Epoch 50, Loss: 1.413145
buy more cat : food : food
Epoch 60, Loss: 1.194449
quick brown cat : jumps : in
Epoch 70, Loss: 1.031466
buy more cat : food : food
Epoch 80, Loss: 0.910929
quick brown cat : jumps : in
Epoch 90, Loss: 0.818127
the dog out : from : its
Epoch 100, Loss: 0.754357
cat stalked a : dust : dust
Epoch 110, Loss: 0.708126
buy more cat : food : food
Epoch 120, Loss: 0.676135
the old cat : hobbled : ,
Epoch 130, Loss: 0.652839
quick brown cat : jumps : jumps
Epoch 140, Loss: 0.638570
fido the cat : would : ,
Epoch 150, Loss: 0.628520
the dog out : from : its
Epoch 160, Loss: 0.621440
cat stalked a : dust : dust
Epoch 170, Loss: 0.616942
this animal is : dog : cat
Epoch 180, Loss: 0.612559
the dog out : from : its
E

# 評価

In [ ]:
# 次の単語を予測
test_dataset.test(model)

cat stalked a : dust : dust


In [ ]:
# 重みの表示
model.embeddings.weight

Parameter containing:
tensor([[-0.8693, -0.6430,  0.9806,  ..., -1.1147, -0.3991,  1.7156],
        [ 1.2227, -3.1026,  0.1475,  ..., -2.2515,  1.6006, -0.6037],
        [-2.3367,  2.5572, -2.6345,  ..., -0.0532,  0.4508,  0.7829],
        ...,
        [-4.6001, -0.7135, -3.9227,  ...,  3.7509,  0.0690,  3.2888],
        [ 4.7064, -3.1684, -3.7181,  ...,  0.5270,  2.7482,  0.7752],
        [-1.6228, -3.6499, -3.0329,  ...,  4.4151,  2.7994, -0.1086]],
       requires_grad=True)